# import

In [1]:
%load_ext autoreload
%reload_ext autoreload
#autoreload 2：装入所有 %aimport 不包含的模块。
%autoreload 2
#%matplotpy
import numpy as np
from scuteegfe.mne_features_wrapper.feature_wrapper import Feature

# test EMD

In [ ]:
rng = np.random.RandomState(42)
n_epochs, n_channels, n_times = 2, 2,250
X = rng.randn(n_epochs, n_channels, n_times)
feat1=Feature(X,sfreq=250,selected_funcs={'decorr_time'})
print(feat1.features.shape)
# feat2=Feature(X,sfreq=250,selected_funcs={'pow_freq_bands'})juju
# print(feat2.features.shape)
select_para=dict({'EMD__EMD_times': 1,
                  'EMD__EMD_params': 6})
feat=Feature(X,sfreq=250,selected_funcs={'EMD'},funcs_params=select_para)
print(feat.features.shape)

# test bicoherence

In [ ]:
#test hos
'''
bicoherence input  Multidimensional  vector [n_y,n_d] 
bicoherence input  one-dimensional   vector [n_y,1]  or [1，n_y] 
'''
import os,sys
from scuteegfe.HOSA.conventional.bicoherence import bicoherence
import scipy.io as scio
load_mat = scio.loadmat('qpc.mat')['zmat']
y=load_mat[0:4,:]
y1=np.transpose(load_mat[0:4,:])
y2=load_mat[0,:][np.newaxis,:]
y3=np.transpose(load_mat[1,:][np.newaxis,:])
bicoherence(y, nfft=None, wind=None, nsamp=None, overlap=None)
bicoherence(y1, nfft=None, wind=None, nsamp=None, overlap=None)
bicoherence(y2, nfft=None, wind=None, nsamp=None, overlap=None)
bicoherence(y3, nfft=None, wind=None, nsamp=None, overlap=None)

# test wavelet_entropy

In [3]:
rng = np.random.RandomState(42)
n_epochs, n_channels, n_times = 2,2,2000
X = rng.randn(n_epochs, n_channels, n_times)
feat=Feature(X,sfreq=250,selected_funcs={'wavelet_entropy'})
print(feat.features.shape)

(2, 2, 7)


# test wavelet_entropy unaverage

In [10]:
import pywt
def imp_extract(section_data,Fs, time_sec,wavelet_name):
    f = np.arange(1, 129, 0.2)
    [wt, f1] = pywt.cwt(section_data, f, wavelet_name, 1 / Fs)  # 'mexh'
    cwt_re = np.sum(abs(wt), axis=1) * 2 / time_sec;  #
    return cwt_re,f1
def band_DE(Pxx, f, Par_ratios=1, band=None):
    """
    Feature extraction of fixed frequency band
    :param Pxx:  frequency band parameter
    :param f:    frequency range
    :param band: selected frequency band
    :return:     固定频带的特征
    """
    fea_num=int(band.shape[0])
    psd = np.empty((fea_num))
    for i in range(fea_num):
        idx = np.where((f >= band[i, 0]) & (f <= band[i, 1]))
        psd[i] = np.sum(np.multiply(Pxx[idx], Pxx[idx]))
        if Par_ratios == 1:
            san_D = np.hstack((psd, psd[2] / psd[1], psd[3] / psd[1]))
        else:
            san_D = psd
    return san_D
rng = np.random.RandomState(42)
n_epochs, n_channels, n_times = 2, 4,2500
data= rng.randn(n_channels, n_times)
sfreq=250
m_times=1
m_Par_ratios=1
m_entropy=True               
wavelet_name= 'gaus1'
Average=False
band=np.array([[2, 3.8], [4, 7], [8, 13], [14, 30], [31, 48]])
time_sec = int(m_times * sfreq)
fea_num = int(band.shape[0] + m_Par_ratios * 2)
n_channel, n_times = data.shape
section_num = int(np.ceil((n_times/ time_sec)))
if Average:
   de = np.empty((int(n_channel),fea_num))
else:
   de = np.empty((int(n_channel),fea_num,section_num))

for channel in range(n_channel):
    # initialization
    section_de = np.empty((fea_num, section_num))
    # for one second calculate cwt
    for section in range(section_num):
        section_data = data[channel, section * time_sec:(section + 1) * time_sec]
        spec, f = imp_extract(section_data=section_data, Fs=sfreq, time_sec=time_sec,wavelet_name=wavelet_name)
        section_de[:, section] = band_DE(spec, f, Par_ratios=m_Par_ratios, band=band)
    if Average:
      de_mean = np.sum(section_de, axis=1);
    else:
      de_mean = section_de; 
    if m_entropy:
        de_mean = np.multiply(de_mean, np.log(de_mean));
    de[channel,:] = de_mean
    print(de.shape)
feature = de.reshape(-1)
print(feature.shape)

(4, 7, 10)
(4, 7, 10)
(4, 7, 10)
(4, 7, 10)
(280,)
